In [1]:
# Use a pipeline as a high-level helper





C:\Users\Mihan Perera\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from statistics import mean 
from datetime import date, timedelta
import requests


# Function to retrieve financial news for a particular stock and date
def get_stock_news(stock_symbol, date):
    
    newsapi_key = 'ade9bf20-9bf7-4875-afdc-a2a71b11ceff'
    base_url = 'https://newsapi.org/v2/everything'
  
    # Date format should be in yyyy-mm-dd
    date_str = date
    
    # Define the parameters for the API request
    params = {
        'apiKey': newsapi_key,
        'q': stock_symbol,
        'from': date_str,
        'to': date_str,
        'language': 'en',  # English language news
    }
     
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if 'articles' in data:
            
            # Retrieve and return the list of news articles
            news_articles = data['articles']
            return news_articles
        else:
            return "No news articles found for this stock and date."
        
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

    

    # Function to perform sentiment analysis on financial news articles
def sentiment_score(symbol, date):
    
    # Initialize an empty list to store sentiment scores
    pipe = pipeline("text-classification", model="ProsusAI/finbert")
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    
    # Initialize an empty list to store sentiment scores
    score = []
    
    # Get the list of news articles using get_stock_news function
    news_articles = get_stock_news(symbol, date)
    
    if isinstance(news_articles, list):
        print(f"Financial news for {stock_symbol} on {date}:")
      
        # Iterate through each news article and perform sentiment analysis
        for index, article in enumerate(news_articles, start=1):
            
            res = pipe(article['title'])
            
            # Append sentiment scores to the list based on sentiment labels
            if res[0]['label'] == 'positive':
                score.append(res[0]['score'])  
            if res[0]['label'] == 'negative':
                score.append(-res[0]['score'])
            if res[0]['label'] == 'neutral':
                score.append(res[0]['score']*0)
     
        return mean(score)

